In [1]:
# Cell 1: Create templates directory
!mkdir -p templates
print("Created 'templates' directory.")

Created 'templates' directory.


In [2]:
%%writefile blockchain.py
import hashlib
import json
from datetime import datetime

class Block:
    def __init__(self, index, timestamp, data, previous_hash):
        self.index = index
        self.timestamp = timestamp
        self.data = data
        self.previous_hash = previous_hash
        self.hash = self.calculate_hash() # This will now call the fixed calculate_hash

    def calculate_hash(self):
        # Data to be hashed should *not* include the hash itself,
        # as the hash is derived from the other data.
        # Create a dictionary of the block's core attributes for hashing
        data_to_hash = {
            'index': self.index,
            'timestamp': self.timestamp.isoformat(), # Use ISO format for consistency
            'data': self.data,
            'previous_hash': self.previous_hash
        }
        # Convert the dictionary to a JSON string, sort keys for consistent hashing
        block_string = json.dumps(data_to_hash, sort_keys=True).encode('utf-8')
        return hashlib.sha256(block_string).hexdigest()

    def to_dict(self):
        # Helper to convert block data to a dictionary for JSON serialization
        # Convert datetime to ISO format string for robust serialization/deserialization
        return {
            'index': self.index,
            'timestamp': self.timestamp.isoformat(), # Use ISO format for datetime
            'data': self.data,
            'previous_hash': self.previous_hash,
            'hash': self.hash # Include hash for easier loading, though it's recalculated for validation
        }

class Blockchain:
    def __init__(self):
        self.chain = []
        # Genesis block is created implicitly if no file is loaded, or explicitly if needed
        # It's usually better to create it only if load_chain_from_file fails or if chain is empty
        # after load.

    def create_genesis_block(self):
        """Creates the first block in the chain."""
        if not self.chain: # Only create if chain is empty
            genesis_data = "Genesis Block - Diabetes Prediction History"
            genesis_block = Block(0, datetime.now(), genesis_data, "0")
            self.chain.append(genesis_block)
            print("Genesis Block created.")

    def add_block(self, block):
        """Adds a new block to the chain after validation."""
        if len(self.chain) > 0:
            last_block_hash = self.chain[-1].hash
            if block.previous_hash != last_block_hash:
                print(f"Error: Invalid previous hash for new block! Expected {last_block_hash}, got {block.previous_hash}")
                return False
            # Re-calculate hash to ensure integrity upon addition
            if block.hash != block.calculate_hash():
                print("Error: Block hash mismatch!")
                return False
        self.chain.append(block)
        return True

    def get_last_block(self):
        """Returns the last block in the chain."""
        if not self.chain:
            self.create_genesis_block() # Ensure genesis block exists if chain is empty
        return self.chain[-1]

    def is_chain_valid(self):
        """Verifies the integrity of the entire chain."""
        for i in range(1, len(self.chain)):
            current_block = self.chain[i]
            previous_block = self.chain[i-1]

            # Check if the current block's hash is correct (recalculate and compare)
            if current_block.hash != current_block.calculate_hash():
                print(f"Chain invalid: Block {current_block.index} hash mismatch.")
                return False
            # Check if the current block points to the correct previous hash
            if current_block.previous_hash != previous_block.hash:
                print(f"Chain invalid: Block {current_block.index} previous hash mismatch.")
                return False
        return True

    def to_list(self):
        """Converts the blockchain to a list of dictionaries for easy display."""
        return [block.to_dict() for block in self.chain]

    def save_chain_to_file(self, filename):
        """Saves the current blockchain to a JSON file."""
        try:
            with open(filename, 'w') as f:
                chain_data = [block.to_dict() for block in self.chain]
                json.dump(chain_data, f, indent=4)
            # print(f"Blockchain saved to {filename}") # Uncomment for verbose saving
        except Exception as e:
            print(f"Error saving blockchain to file {filename}: {e}")

    def load_chain_from_file(self, filename):
        """Loads a blockchain from a JSON file, rebuilding the Block objects."""
        try:
            with open(filename, 'r') as f:
                chain_data = json.load(f)

            new_chain = []
            for block_data in chain_data:
                # Ensure timestamp is converted back to datetime object from ISO string
                block_data['timestamp'] = datetime.fromisoformat(block_data['timestamp'])

                # Reconstruct Block object. The Block.__init__ will recalculate 'hash'
                # to ensure integrity, so we can ignore the 'hash' key from block_data
                # when passing to the constructor.
                loaded_hash = block_data.pop('hash', None)

                block = Block(**block_data)

                # OPTIONAL: Validate loaded_hash vs recalculated hash for extra security during load
                if loaded_hash and loaded_hash != block.hash:
                    print(f"Integrity warning: Block {block.index} hash mismatch during load.")

                new_chain.append(block)

            self.chain = new_chain
            print(f"Blockchain loaded from {filename}. Total blocks: {len(self.chain)}")
            if not self.is_chain_valid():
                print("Warning: Loaded blockchain is not valid! Potential tampering or corruption.")
                return False # Indicate loading failed due to invalidity
            return True # Indicate successful load
        except FileNotFoundError:
            print(f"Blockchain file '{filename}' not found. Will start a new chain.")
            return False # Indicate file not found, app should create new chain
        except json.JSONDecodeError:
            print(f"Error: Blockchain file '{filename}' is corrupted (JSONDecodeError). Starting a new chain.")
            return False
        except Exception as e:
            print(f"An unexpected error occurred loading blockchain from '{filename}': {e}. Starting a new chain.")
            return False

Writing blockchain.py


In [3]:
%%writefile templates/history.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Blockchain History</title>
    <script src="https://cdn.tailwindcss.com"></script>
    <style>
        body {
            font-family: 'Inter', sans-serif;
            background-color: #f3f4f6; /* Light gray background */
            color: #374151;
        }
        .container {
            max-width: 1000px;
            margin: 40px auto;
            padding: 30px;
            background-color: #ffffff;
            border-radius: 1.5rem;
            box-shadow: 0 10px 20px rgba(0, 0, 0, 0.1);
        }
        th, td {
            padding: 0.75rem 1rem;
            text-align: left;
            border-bottom: 1px solid #e5e7eb; /* Light border between rows */
        }
        th {
            background-color: #f9fafb; /* Very light gray for header */
            font-weight: 600;
            color: #1f2937; /* Darker text for header */
            text-transform: uppercase;
            font-size: 0.875rem; /* Smaller font for header */
            letter-spacing: 0.05em;
        }
        tr:hover {
            background-color: #f1f5f9; /* Lightest gray on hover for rows */
        }
        .valid-badge {
            background-color: #d1fae5; /* Light green */
            color: #065f46; /* Dark green text */
            padding: 0.25rem 0.75rem;
            border-radius: 0.5rem;
            font-weight: 600;
            font-size: 0.875rem;
        }
        .invalid-badge {
            background-color: #fee2e2; /* Light red */
            color: #991b1b; /* Dark red text */
            padding: 0.25rem 0.75rem;
            border-radius: 0.5rem;
            font-weight: 600;
            font-size: 0.875rem;
        }
    </style>
</head>
<body class="bg-gray-100 p-4">
    <div class="container">
        <h1 class="text-4xl font-extrabold text-center text-blue-600 mb-6">
            Prediction History Blockchain
        </h1>

        <div class="mb-8 text-center text-lg">
            <p class="font-semibold text-gray-700">
                Blockchain Validity:
                <span class="{{ 'valid-badge' if is_valid else 'invalid-badge' }}">
                    {{ 'Valid' if is_valid else 'Invalid' }}
                </span>
            </p>
            <p class="text-gray-600">Total Blocks: {{ chain|length }}</p>
        </div>

        <div class="overflow-x-auto">
            <table class="min-w-full bg-white shadow-md rounded-lg overflow-hidden">
                <thead>
                    <tr>
                        <th>Index</th>
                        <th>Timestamp</th>
                        <th>Input Hash</th>
                        <th>Prediction</th>
                        <th>Confidence</th>
                        <th>Previous Hash</th>
                        <th>Block Hash</th>
                    </tr>
                </thead>
                <tbody>
                    {% for block in chain %}
                    <tr>
                        <td>{{ block.index }}</td>
                        <td>{{ block.timestamp }}</td>
                        <td>
                            {% if block.data.input_form_hash %}
                                <span title="{{ block.data.input_form_hash }}">
                                    {{ block.data.input_form_hash[:8] }}...
                                </span>
                            {% else %}
                                N/A {# For Genesis Block or if data structure is different #}
                            {% endif %}
                        </td>
                        <td>
                            {% if block.data.prediction_stage %}
                                <span class="font-bold text-{{ 'green-600' if block.data.prediction_stage == 'N' else ('yellow-600' if block.data.prediction_stage == 'P' else 'red-600') }}">
                                    {{ block.data.prediction_stage }}
                                </span>
                            {% else %}
                                {{ block.data }} {# For Genesis Block or if 'data' is simple string #}
                            {% endif %}
                        </td>
                        <td>
                            {% if block.data.confidence_score %}
                                {{ block.data.confidence_score }}%
                            {% else %}
                                N/A
                            {% endif %}
                        </td>
                        <td>
                            <span title="{{ block.previous_hash }}">
                                {{ block.previous_hash[:8] }}...
                            </span>
                        </td>
                        <td>
                            <span title="{{ block.hash }}">
                                {{ block.hash[:8] }}...
                            </span>
                        </td>
                    </tr>
                    {% endfor %}
                </tbody>
            </table>
        </div>
        <div class="text-center mt-8">
            <a href="/" class="inline-block bg-blue-500 hover:bg-blue-600 text-white font-bold py-2 px-4 rounded-lg transition duration-300">
                &larr; Back to Prediction
            </a>
        </div>
    </div>
</body>
</html>

Writing templates/history.html


In [4]:
%%writefile app.py
from flask import Flask, render_template, request, jsonify
import joblib
import numpy as np
import pandas as pd
import os
from datetime import datetime
import hashlib
import json

# --- Import your Blockchain class ---
from blockchain import Block, Blockchain
# -----------------------------------

app = Flask(__name__)

# Define the directory where your model components are saved in Google Drive
MODEL_DIR = '/content/drive/MyDrive/Diabetic_Prediction_Model'
# --- Define the path for your blockchain persistence file ---
BLOCKCHAIN_FILE = os.path.join(MODEL_DIR, 'diabetes_blockchain.json')
# -------------------------------------------------------------

# Load the trained model components
ensemble_model = None
scaler = None
gender_encoder = None
selected_columns = None
original_feature_names = None

try:
    ensemble_model = joblib.load(os.path.join(MODEL_DIR, 'ensemble_model.pkl'))
    scaler = joblib.load(os.path.join(MODEL_DIR, 'scaler.pkl'))
    gender_encoder = joblib.load(os.path.join(MODEL_DIR, 'gender_encoder.pkl'))
    selected_columns = joblib.load(os.path.join(MODEL_DIR, 'selected_columns.pkl'))
    original_feature_names = joblib.load(os.path.join(MODEL_DIR, 'original_feature_names.pkl'))
    print("All model components loaded successfully!")
except FileNotFoundError as e:
    print(f"Error loading model components: {e}")
    print(f"Please ensure your model files are in {MODEL_DIR} and Drive is mounted.")

# --- Initialize your Blockchain instance ---
d_blockchain = Blockchain()
# Try to load existing chain, otherwise create genesis block
if not d_blockchain.load_chain_from_file(BLOCKCHAIN_FILE):
    d_blockchain.create_genesis_block()
print(f"Blockchain initialized. Total blocks: {len(d_blockchain.chain)}")
# -----------------------------------------

# Mapping for display
PREDICTION_MAP = {0: 'N', 1: 'P', 2: 'Y'}
FULL_PREDICTION_MAP = {'N': 'Non-Diabetic', 'P': 'Pre-Diabetic', 'Y': 'Diabetic'}

# Recommendations based on predicted stage
RECOMMENDATIONS = {
    'N': [
        "Maintain a balanced diet with plenty of fruits, vegetables, and whole grains.",
        "Engage in regular physical activity (at least 30 minutes most days of the week).",
        "Monitor your weight and maintain a healthy BMI.",
        "Stay hydrated and limit sugary drinks.",
        "Get regular check-ups with your doctor."
    ],
    'P': [
        "Cut down significantly on sugar and processed foods.",
        "Increase your exercise and monitor your weight closely.",
        "Check blood sugar levels regularly as advised by a doctor.",
        "Consider consulting a nutritionist for a personalized meal plan.",
        "Reduce stress and ensure adequate sleep."

    ],
    'Y': [
        "Strictly follow your doctor's treatment plan and medication schedule.",
        "Adhere to a diabetic-friendly diet, focusing on low glycemic index foods.",
        "Regularly monitor blood sugar levels and keep a log.",
        "Engage in consistent physical activity, as approved by your doctor.",
        "Attend all scheduled medical appointments and screenings.",
        "Educate yourself on diabetes management and potential complications."
    ]
}

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():
    if ensemble_model is None or scaler is None or gender_encoder is None or \
       selected_columns is None or original_feature_names is None:
        return render_template('index.html', prediction_result="Error", confidence_score="N/A",
                               recommendations=["Model components not loaded. Please check server logs and file paths."])

    try:
        data = request.form.to_dict()
        print(f"DEBUG: Raw form data: {data}")

        input_values = {}
        for key, value in data.items():
            if key in ['AGE', 'Urea', 'Cr', 'HbA1c', 'Chol', 'TG', 'HDL', 'LDL', 'VLDL', 'BMI']:
                input_values[key] = float(value)
            else:
                input_values[key] = value
        print(f"DEBUG: Processed input_values (after unit conversion fix): {input_values}")

        processed_input = {}
        for feature in original_feature_names:
            if feature == 'Gender':
                gender_val = input_values.get('Gender', 'F')
                processed_input[feature] = gender_encoder.transform([str(gender_val).upper()])[0]
            elif feature in input_values:
                processed_input[feature] = input_values[feature]
            else:
                processed_input[feature] = 0
        print(f"DEBUG: processed_input before DataFrame: {processed_input}")

        input_for_scaling = pd.DataFrame([processed_input], columns=original_feature_names)
        print(f"DEBUG: input_for_scaling DataFrame:\n{input_for_scaling}")

        scaled_input = scaler.transform(input_for_scaling)
        print(f"DEBUG: scaled_input (after scaler.transform):\n{scaled_input}")

        final_input = scaled_input[:, selected_columns]
        print(f"DEBUG: final_input (after feature selection):\n{final_input}")
        print(f"DEBUG FINAL INPUT SHAPE: {final_input.shape}")
        print(f"DEBUG FINAL INPUT SAMPLE: {final_input[0]}")

        prediction_numeric = ensemble_model.predict(final_input)[0]
        prediction_proba = ensemble_model.predict_proba(final_input)[0]

        print(f"DEBUG: Ensemble Model Probabilities (N=0, P=1, Y=2): {prediction_proba}")

        # --- START NEW, AGGRESSIVE QUICK FIX FOR PRE-DIABETIC (P) CLASSIFICATION ---
        hba1c_input = input_values.get('HbA1c')

        HBA1C_PREDIABETES_START = 5.7
        HBA1C_PREDIABETES_END = 6.4

        # If HbA1c is within the *medical* pre-diabetic range,
        # FORCE the prediction to be Pre-Diabetic (P = 1).
        if hba1c_input is not None and HBA1C_PREDIABETES_START <= hba1c_input <= HBA1C_PREDIABETES_END:
            prediction_numeric = 1 # Force to P
            print(f"DEBUG: Aggressive Quick Fix Applied: HbA1c ({hba1c_input}) is in P range. Forced prediction to P.")
        # --- END NEW, AGGRESSIVE QUICK FIX ---

        confidence = np.max(prediction_proba) * 100 # Confidence still based on original probas for display
        prediction_stage = PREDICTION_MAP.get(prediction_numeric, 'Unknown')
        prediction_full_text = FULL_PREDICTION_MAP.get(prediction_stage, 'Unknown Stage')
        current_recommendations = RECOMMENDATIONS.get(prediction_stage, ["No specific recommendations available."])

        # --- BLOCKCHAIN INTEGRATION START ---
        raw_input_data_for_hash = {k: v for k, v in sorted(data.items())}
        input_hash_val = hashlib.sha256(json.dumps(
            raw_input_data_for_hash, sort_keys=True
        ).encode('utf-8')).hexdigest()

        prediction_record = {
            "input_form_hash": input_hash_val,
            "prediction_numeric": int(prediction_numeric),
            "prediction_stage": prediction_stage,
            "confidence_score": float(f"{confidence:.2f}"),
            "timestamp_prediction": str(datetime.now())
        }

        last_block = d_blockchain.get_last_block()
        new_block_index = last_block.index + 1
        new_block = Block(
            new_block_index,
            datetime.now(),
            prediction_record,
            last_block.hash
        )
        if d_blockchain.add_block(new_block):
            print(f"Prediction recorded to blockchain. New block index: {new_block_index}")
            d_blockchain.save_chain_to_file(BLOCKCHAIN_FILE)
        else:
            print(f"Failed to add block {new_block_index} to blockchain. Chain might be invalid.")
        # --- BLOCKCHAIN INTEGRATION END ---

        return render_template('index.html',
                               prediction_result=prediction_stage,
                               prediction_result_full_text=prediction_full_text,
                               confidence_score=f"{confidence:.2f}",
                               recommendations=current_recommendations)

    except Exception as e:
        error_message = f"An error occurred during prediction: {e}. Please check your input values and server logs."
        print(error_message)
        return render_template('index.html', prediction_result="Error", confidence_score="N/A",
                               recommendations=[error_message])

@app.route('/history', methods=['GET'])
def view_history():
    chain_list = d_blockchain.to_list()
    is_valid = d_blockchain.is_chain_valid()
    return render_template('history.html', chain=chain_list, is_valid=is_valid)

@app.route('/blockchain_raw', methods=['GET'])
def get_blockchain_raw():
    chain_list = d_blockchain.to_list()
    is_valid = d_blockchain.is_chain_valid()
    return jsonify({
        'chain': chain_list,
        'length': len(chain_list),
        'is_valid': is_valid
    })

if __name__ == '__main__':
    pass

Writing app.py


In [5]:
#import os
#BLOCKCHAIN_FILE = '/content/drive/MyDrive/Diabetic_Prediction_Model/diabetes_blockchain.json'
#
#if os.path.exists(BLOCKCHAIN_FILE):
#    os.remove(BLOCKCHAIN_FILE)
#    print(f"Removed existing blockchain file: {BLOCKCHAIN_FILE}")
#else:
#    print(f"Blockchain file not found at {BLOCKCHAIN_FILE}. Nothing to remove.")

In [ ]:

!pip install Flask numpy pandas scikit-learn joblib pyngrok

# Mount Google Drive (if not already mounted)
from google.colab import drive
drive.mount('/content/drive')

# --- Define your project folder path on Google Drive ---
PROJECT_DIR_ON_DRIVE = '/content/drive/MyDrive/Diabetic_Prediction_Model'

# Create the project directory on Drive if it doesn't exist
!mkdir -p "$PROJECT_DIR_ON_DRIVE"

# --- Copy all necessary application files and folders from /content/ to your Drive project directory ---
!cp /content/app.py "$PROJECT_DIR_ON_DRIVE/app.py"
!cp /content/blockchain.py "$PROJECT_DIR_ON_DRIVE/blockchain.py"
!cp -r /content/templates "$PROJECT_DIR_ON_DRIVE/"

# --- Now, change directory to your project folder on Drive ---
import os
os.chdir(PROJECT_DIR_ON_DRIVE)


# --- Ngrok setup ---
from pyngrok import ngrok

# Authenticate ngrok (get your auth token from https://dashboard.ngrok.com/get-started/your-authtoken)
# >>> IMPORTANT: YOU MUST UNCOMMENT THE NEXT LINE AND REPLACE "YOUR_NGROK_AUTH_TOKEN_HERE" WITH YOUR ACTUAL TOKEN <<<
ngrok.set_auth_token("2zyJnOxcmtm1Co8YdYsAdxgr4ie_28REAYU6P1syhoaA4ey2R") # <--- PASTE YOUR NGROK AUTH TOKEN HERE!

# Start ngrok tunnel, targeting port 8000
public_url = ngrok.connect(8000)
print(f"Flask App Public URL: {public_url}")

# Run your Flask app on port 8000
!python -m flask run --host=0.0.0.0 --port=8000

Mounted at /content/drive
Flask App Public URL: NgrokTunnel: "https://400210e616d4.ngrok-free.app" -> "http://localhost:8000"
All model components loaded successfully!
Blockchain loaded from /content/drive/MyDrive/Diabetic_Prediction_Model/diabetes_blockchain.json. Total blocks: 10
Blockchain initialized. Total blocks: 10
 * Debug mode: off
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8000
 * Running on http://172.28.0.12:8000
Press CTRL+C to quit
127.0.0.1 - - [18/Jul/2025 21:08:48] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Jul/2025 21:08:50] "GET /favicon.ico HTTP/1.1" 404 -


In [ ]:
!cat app.py